In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import r2_score, mean_absolute_error

# List of CSV files
csv_files = [
    "dry blub temperature.csv",
    "potential temperature.csv",
    "sea surface temperature.csv",
    "surface heigh.csv",
    "surface_height.csv",
    "vertical velocity at t points.csv"
]

# Dictionary to store DataFrames
dfs = {}

# Load datasets
for file in csv_files:
    if os.path.exists(file):
        try:
            df = pd.read_csv(file, delimiter=",", encoding="utf-8", on_bad_lines="skip", engine="python", skiprows=8)  # Skip metadata

            # Convert all columns (except datetime) to numeric, forcing errors to NaN
            for col in df.columns:
                if col not in ["DATETIME", "TIME"]:  # Exclude time-related columns
                    df[col] = pd.to_numeric(df[col], errors="coerce")

            # Replace -1.E+34 (error values) with NaN
            df.replace([-1e+34, "-1.E+34", -1.0e+34, "-1.000000e+34"], np.nan, inplace=True)

            dfs[file] = df
            print(f"✅ Loaded {file} successfully! Shape: {df.shape}")
        except Exception as e:
            print(f"❌ Error loading {file}: {e}")
    else:
        print(f"⚠️ Warning: {file} not found!")

# Function to perform LSTM on each dataset
def perform_lstm(dfs):
    for file, df in dfs.items():
        print(f"\n📂 **Performing LSTM on {file}**")

        # Select numeric columns
        numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns

        if not numeric_cols.empty:
            # Scale data
            scaler = MinMaxScaler()
            scaled_data = scaler.fit_transform(df[numeric_cols])

            # Reshape data for LSTM
            sequence_length = 10
            X = []
            y = []
            for i in range(sequence_length, len(scaled_data)):
                X.append(scaled_data[i-sequence_length:i])
                y.append(scaled_data[i])

            X = np.array(X)
            y = np.array(y)

            # Split data into training and testing sets
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=False)

            # Reshape for LSTM input
            X_train = X_train.reshape(X_train.shape[0], sequence_length, scaled_data.shape[1])
            X_test = X_test.reshape(X_test.shape[0], sequence_length, scaled_data.shape[1])

            # Build LSTM model
            model = Sequential([
                LSTM(units=50, return_sequences=True, input_shape=(sequence_length, scaled_data.shape[1])),
                LSTM(units=50),
                Dense(scaled_data.shape[1])
            ])

            # Compile model
            model.compile(optimizer='adam', loss='mse')

            # Train model
            model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))

            # Evaluate model
            loss = model.evaluate(X_test, y_test)
            print(f"✅ Test Loss: {loss}")
        else:
            print(f"⚠️ No numeric columns found in {file}.")

# Perform LSTM on each dataset
perform_lstm(dfs)


# Predict on the test set
y_pred = model.predict(X_test)

# Compute R² Score and MAE
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# Print results
print(f"✅ Test Loss (MSE): {loss:.4f}")
print(f"📊 R² Score: {r2:.4f}")
print(f"📉 Mean Absolute Error (MAE): {mae:.4f}")


✅ Loaded dry blub temperature.csv successfully! Shape: (145, 1)
✅ Loaded potential temperature.csv successfully! Shape: (17473, 1)
✅ Loaded sea surface temperature.csv successfully! Shape: (20384, 5)
✅ Loaded surface heigh.csv successfully! Shape: (20384, 5)
✅ Loaded surface_height.csv successfully! Shape: (20384, 5)
✅ Loaded vertical velocity at t points.csv successfully! Shape: (17473, 1)

📂 **Performing LSTM on dry blub temperature.csv**


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 195ms/step - loss: nan - val_loss: nan
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: nan - val_loss: nan
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: nan - val_loss: nan
Epoch 4/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: nan - val_loss: nan
Epoch 5/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: nan - val_loss: nan
Epoch 6/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: nan - val_loss: nan
Epoch 7/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: nan - val_loss: nan
Epoch 8/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: nan - val_loss: nan
Epoch 9/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: nan - val_loss: nan
Epoch 10/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: nan - val_loss: nan
Epoch 11/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: nan - val_loss: nan
Epoch 12/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: nan - val_loss: nan
Epoch 13/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: nan - val_loss

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


764/764 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: nan - val_loss: nan
Epoch 2/50
764/764 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - loss: nan - val_loss: nan
Epoch 3/50
764/764 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: nan - val_loss: nan
Epoch 4/50
764/764 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: nan - val_loss: nan
Epoch 5/50
764/764 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: nan - val_loss: nan
Epoch 6/50
764/764 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: nan - val_loss: nan
Epoch 7/50
764/764 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: nan - val_loss: nan
Epoch 8/50
764/764 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: nan - val_loss: nan
Epoch 9/50
764/764 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: nan - val_loss: nan
Epoch 10/50
764/764 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: nan - val_loss: nan
Epoch 11/50
764/764 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: nan - val_loss: nan
Epoch 12/50
764/764 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: nan - val_loss: nan
Epoch 13/50
764/764 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms

...

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


892/892 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - loss: nan - val_loss: nan
Epoch 2/50
892/892 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: nan - val_loss: nan
Epoch 3/50
892/892 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - loss: nan - val_loss: nan
Epoch 4/50
892/892 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: nan - val_loss: nan
Epoch 5/50
892/892 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - loss: nan - val_loss: nan
Epoch 6/50
892/892 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: nan - val_loss: nan
Epoch 7/50
892/892 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: nan - val_loss: nan
Epoch 8/50
892/892 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: nan - val_loss: nan
Epoch 9/50
892/892 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: nan - val_loss: nan
Epoch 10/50
892/892 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: nan - val_loss: nan
Epoch 11/50
892/892 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - loss: nan - val_loss: nan
Epoch 12/50
892/892 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - loss: nan - val_loss: nan
Epoch 13/50
892/892 ━━━━━━━━━━━━━━━━━━━━ 6s 7

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


764/764 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: nan - val_loss: nan
Epoch 2/50
764/764 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: nan - val_loss: nan
Epoch 3/50
764/764 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: nan - val_loss: nan
Epoch 4/50
764/764 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: nan - val_loss: nan
Epoch 5/50
764/764 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: nan - val_loss: nan
Epoch 6/50
764/764 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: nan - val_loss: nan
Epoch 7/50
764/764 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: nan - val_loss: nan
Epoch 8/50
764/764 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: nan - val_loss: nan
Epoch 9/50
764/764 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: nan - val_loss: nan
Epoch 10/50
764/764 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: nan - val_loss: nan
Epoch 11/50
764/764 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: nan - val_loss: nan
Epoch 12/50
764/764 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: nan - val_loss: nan
Epoch 13/50
764/764 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/

NameError: name 'model' is not defined